In [1]:
import pandas as pd
import numpy as np
# from sklearn.impute import KNNImputer

# Load the dataset
df = pd.read_csv("D:/training_loan_data.csv", header=1)
df.head()

,id,member_id,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,desc,purpose,...,inq_last_6mths,mths_since_recent_inq,revol_util,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,application_approved_flag,internal_score,bad_flag
0,10000001,11983056.0,7550,36 months,16.24%,3 years,RENT,28000.0,NaN,debt_consolidation,...,0.0,17.0,72%,4000.0,NaN,3828.953801,5759.0,1,99,0.0
1,10000002,12002921.0,27050,36 months,10.99%,10+ years,OWN,55000.0,Borrower added on 12/31/13 > Combining high ...,debt_consolidation,...,0.0,8.0,61.20%,35700.0,NaN,34359.940730,114834.0,1,353,0.0
2,10000003,11983096.0,12000,36 months,10.99%,4 years,RENT,60000.0,Borrower added on 12/31/13 > I would like to...,debt_consolidation,...,1.0,3.0,24%,18100.0,NaN,16416.617760,7137.0,1,157,0.0
3,10000004,12003142.0,28000,36 months,7.62%,5 years,MORTGAGE,325000.0,NaN,debt_consolidation,...,1.0,3.0,54.60%,42200.0,NaN,38014.149760,799592.0,1,365,0.0
4,10000005,11993233.0,12000,36 months,13.53%,10+ years,RENT,40000.0,NaN,debt_consolidation,...,0.0,17.0,68.80%,7000.0,53.0,6471.462236,13605.0,1,157,0.0


In [2]:
# Inspect the dataset structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199121 entries, 0 to 199120
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           199121 non-null  int64  
 1   member_id                    189457 non-null  float64
 2   loan_amnt                    199121 non-null  int64  
 3   term                         189457 non-null  object 
 4   int_rate                     189457 non-null  object 
 5   emp_length                   181531 non-null  object 
 6   home_ownership               189457 non-null  object 
 7   annual_inc                   189457 non-null  float64
 8   desc                         82004 non-null   object 
 9   purpose                      189457 non-null  object 
 10  percent_bc_gt_75             180419 non-null  float64
 11  bc_util                      180333 non-null  float64
 12  dti                          189457 non-null  float64
 13 

In [3]:
# Function to count null values and calculate the percentage of nulls
def NullCheck(data):
    d = dict()
    for col in data.columns:
        d[col] = [data[col].isnull().sum(), round(data[col].isnull().mean()*100,3)]
    ddd = pd.DataFrame(d).T
    return ddd.rename(columns={0: "NullSum", 1: "NullMean%"})

NullCheck(df)

,NullSum,NullMean%
id,0.0,0.000
member_id,9664.0,4.853
loan_amnt,0.0,0.000
term,9664.0,4.853
int_rate,9664.0,4.853
emp_length,17590.0,8.834
home_ownership,9664.0,4.853
annual_inc,9664.0,4.853
desc,117117.0,58.817
purpose,9664.0,4.853


In [4]:
# Drop rows with missing 'member_id' values
df = df.dropna(subset=['member_id'])

# Drop columns with more than the threshold percentage of NaN values
missing_percentage = round(df.isnull().mean() * 100, 2)
threshold = 50
df = df.drop(columns=missing_percentage[missing_percentage > threshold].index)

# Function to convert columns to numeric values after removing "%" symbols
def mycolconvertper (data, collst):
    for col in collst:
        df[col] = df[col].replace({'%': ''}, regex=True).astype(float)
    return df
# List of columns to convert
lst = ['int_rate','revol_util']
df = mycolconvertper(df, lst)

# Function to extract digits and convert to the specified type
def mycolconvert(data, coldic):
    for column, typ in coldic.items():
        if typ == int:
            data[column] = data[column].str.extract( r'(\d+)').round().astype(typ)
        else:
            data[column] = data[column].str.extract( r'(\d+)').astype(typ)      
    return df
# Column type conversions
dic = {'term': int, 'emp_length': float}
df = mycolconvert(df, dic)

# imputer = KNNImputer(n_neighbors=2)
# df['emp_length'] = imputer.fit_transform(df[['emp_length']])
# df['emp_length'] = df['emp_length'].round().astype(int)

# Function to impute missing values based on skewness
def myimpute(data, collst):
    for col in collst:
        if data[col].dtype in [np.float64, np.int64]:
            skewness = data[col].skew()
            if abs(skewness) > 0.5:
                # High skewness, use median
                print(f"Column '{col}' has high skewness ({skewness:.2f}). Imputing with median.")
                data[col].fillna(data[col].median(), inplace=True)
            else:
                # Low skewness, use mean
                print(f"Column '{col}' has low skewness ({skewness:.2f}). Imputing with mean.")
                data[col].fillna(data[col].mean(), inplace=True)
        else:
            print(f"Column '{col}' is not numerical, skipping imputation.")
    return df

# Identify columns with missing values
dfnch = NullCheck(df)
lst = list(dfnch[dfnch['NullMean%'] != 0].index)
df = myimpute(df, lst)

Column 'emp_length' has low skewness (-0.20). Imputing with mean.
Column 'percent_bc_gt_75' has low skewness (-0.12). Imputing with mean.
Column 'bc_util' has high skewness (-0.69). Imputing with median.
Column 'mths_since_recent_inq' has high skewness (0.88). Imputing with median.
Column 'revol_util' has low skewness (-0.39). Imputing with mean.
Column 'total_bc_limit' has high skewness (2.62). Imputing with median.
Column 'tot_hi_cred_lim' has high skewness (2.64). Imputing with median.
Column 'tot_cur_bal' has high skewness (3.50). Imputing with median.


In [5]:
# Perform one-hot encoding on columns
df = pd.get_dummies(df, columns=['home_ownership'], drop_first=False)
df = pd.get_dummies(df, columns=['purpose'], drop_first=False)

# Display the first 5 rows to check the result
df.head()

,id,member_id,loan_amnt,term,int_rate,emp_length,annual_inc,percent_bc_gt_75,bc_util,dti,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,10000001,11983056.0,7550,36,16.24,3.0,28000.0,100.0,96.0,8.40,...,0,0,0,0,0,0,0,0,0,0
1,10000002,12002921.0,27050,36,10.99,10.0,55000.0,25.0,53.9,22.87,...,0,0,0,0,0,0,0,0,0,0
2,10000003,11983096.0,12000,36,10.99,4.0,60000.0,0.0,15.9,4.62,...,0,0,0,0,0,0,0,0,0,0
3,10000004,12003142.0,28000,36,7.62,5.0,325000.0,16.7,67.1,18.55,...,0,0,0,0,0,0,0,0,0,0
4,10000005,11993233.0,12000,36,13.53,10.0,40000.0,33.3,79.6,16.94,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Check the data types after encoding
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189457 entries, 0 to 199120
Data columns (total 37 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          189457 non-null  int64  
 1   member_id                   189457 non-null  float64
 2   loan_amnt                   189457 non-null  int64  
 3   term                        189457 non-null  int32  
 4   int_rate                    189457 non-null  float64
 5   emp_length                  189457 non-null  float64
 6   annual_inc                  189457 non-null  float64
 7   percent_bc_gt_75            189457 non-null  float64
 8   bc_util                     189457 non-null  float64
 9   dti                         189457 non-null  float64
 10  inq_last_6mths              189457 non-null  float64
 11  mths_since_recent_inq       189457 non-null  float64
 12  revol_util                  189457 non-null  float64
 13  total_bc_limit

In [7]:
df.duplicated(subset=['id']).sum()

1334

In [8]:
df = df.drop_duplicates(subset=['id'], keep='first')
df.duplicated().sum()

0

In [9]:
# Save the DataFrame to a CSV file, excluding the index column
file_path = 'D:/cleaned_training_loan_data.csv'
df.to_csv(file_path, index=False)  